In [8]:
from sklearn.neural_network import MLPClassifier as mlpc
from sklearn.preprocessing import StandardScaler,QuantileTransformer
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, roc_auc_score

import mltools as ml

In [9]:
X = np.genfromtxt('data/X_train.txt', delimiter=',')
Y = np.genfromtxt('data/Y_train.txt', delimiter=',')
np.random.seed(0)

X,Y = ml.shuffleData(X,Y)
X = X[:,:41]

In [10]:
scaler = StandardScaler()
transformer = QuantileTransformer()

Xtr, Xva, Ytr, Yva = ml.splitData(X,Y,0.7)
Xtr_scaled = scaler.fit_transform(Xtr, Ytr)
Xva_scaled = scaler.fit_transform(Xva,Yva)
Xtr_transformed = transformer.fit_transform(Xtr)
Xva_transformed = transformer.fit_transform(Xva)

In [ ]:
learner = mlpc(max_iter=10000)#no weights first.
learner.fit(Xtr_transformed,Ytr) #the training

In [7]:
Yva_hat = learner.predict(Xva_transformed)

print(f" auc: {roc_auc_score(Yva, Yva_hat)} | mse: {mean_squared_error(Yva, Yva_hat)}")

 auc: 1.0 | mse: 0.0
